In [1]:
# Installing libraries
!pip install datasets pandas boto3

  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached numpy-2.2.6-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pyarrow-20.0.0-cp313-cp313-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached aiohttp-3.11.18-cp313-cp313-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.6.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (16 kB)
  Using cached multidict-6.4.4-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.3 kB)
  Using cached propcache-0.3.1-cp313-cp313-m

In [2]:
# Import required libraries
import pandas as pd
from datasets import load_dataset
import boto3
import json
from pathlib import Path

In [3]:
# Read the dataset
df = pd.read_parquet("hf://datasets/reciprocate/vicuna-fair-eval/data/train-00000-of-00001-29a6ee7f79b8544c.parquet")
df.head()

,prompt,selected,rejected
0,USER: How can I improve my time management ski...,Here are some tips to improve your time manag...,Improving your time management skills can hel...
1,USER: What are the main differences between Py...,Python and JavaScript are both popular progra...,Python and JavaScript are both popular progra...
2,USER: How can I increase my productivity while...,Working from home can be both a blessing and ...,Here are some tips that can help you increase...
3,USER: Can you explain the basics of quantum co...,Quantum computing is a type of computing that...,Certainly! Quantum computing is a type of com...
4,USER: What are the differences between plant-b...,Plant-based protein sources are derived from ...,Plant-based protein sources are derived from ...


In [6]:
# Change columns
df = df.rename(columns={'selected': 'referenceResponse'})
df = df.drop('rejected', axis=1)
df.head()

,prompt,referenceResponse
0,USER: How can I improve my time management ski...,Here are some tips to improve your time manag...
1,USER: What are the main differences between Py...,Python and JavaScript are both popular progra...
2,USER: How can I increase my productivity while...,Working from home can be both a blessing and ...
3,USER: Can you explain the basics of quantum co...,Quantum computing is a type of computing that...
4,USER: What are the differences between plant-b...,Plant-based protein sources are derived from ...


In [7]:
# Save as JSONL
output_file = 'processed_data.jsonl'
df.to_json(output_file, orient='records', lines=True)
print(f"Data saved successfully to {output_file}")

Data saved successfully to processed_data.jsonl


In [9]:
# Upload to S3
s3_client = boto3.client('s3')
bucket_name = '<your-backet-name>'
s3_key = 'processed_data.jsonl'
try:
    s3_client.upload_file(str(output_file), bucket_name, s3_key)
    print(f"File uploaded successfully to s3://{bucket_name}/{s3_key}")
except Exception as e:
    print(f"Error uploading file: {str(e)}")

File uploaded successfully
